In [13]:
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *

In [14]:
from sklearn.metrics import confusion_matrix, f1_score


In [15]:


length = 277
DS1_data=pd.read_csv('./data/train_test/DS1_data.csv',header=None).values
DS2_data=pd.read_csv('./data/train_test/DS2_data.csv',header=None).values



In [16]:
train_patients = DS1_data
test_patients = DS2_data


from sklearn.utils import resample

N = train_patients[train_patients[:,-2]==1.0]
L = train_patients[train_patients[:,-2]==2.0]
R = train_patients[train_patients[:,-2]==3.0]
V = train_patients[train_patients[:,-2]==4.0]
A = train_patients[train_patients[:,-2]==5.0]
F = train_patients[train_patients[:,-2]==6.0]


seed=26
np.random.seed(seed)
def downsample(arr, n, seed):
    downsampled = resample(arr,replace=False,n_samples=n, random_state=seed)
    return downsampled
  
def upsample(arr, n, seed):
    upsampled = resample(arr,replace=True,n_samples=n,random_state=seed)
    return upsampled

all_class = [N,L,R,V,A,F]
abn_class = [L,R,V,A,F]

In [17]:
N.shape, L.shape, R.shape, V.shape, A.shape, F.shape,

((58952, 277), (2493, 277), (6725, 277), (2923, 277), (1277, 277), (425, 277))

In [18]:

mean_val = np.mean([len(i) for i in abn_class], dtype= int)
train_sampled = []

for i in abn_class:
    if i.shape[0]> mean_val:
        i = downsample(i,mean_val,seed)
    elif i.shape[0]< mean_val:
        i = upsample(i, mean_val,seed)
    train_sampled.append(i)
    
train_sampled = np.concatenate(train_sampled)



In [19]:
f1_array = []
for x in np.arange(1, 10.1, 0.5):   
    downsampled_data = downsample(all_class[0],int(x * mean_val),seed)
    train_sample = np.concatenate((train_sampled, downsampled_data), axis=0)
    np.random.shuffle(train_sample)
    train_sampled_all = train_sample

    train_values = train_sampled_all
    test_values = pd.read_csv('./data/train_test/test_patients.csv').values

    # Separate the training and testing data, and one-hot encode Y:
    X_train = train_values[:,:-2]
    X_test = test_values[:,:-2]
    y_train = train_values[:,-2]
    y_test = test_values[:,-2]
    X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
    y_test1=to_categorical(y_test)


    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42,stratify=y_train)

    X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
    y_train1=to_categorical(y_train)

    X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
    y_val1=to_categorical(y_val)


    def mlpmodel():
        mlpmodel = Sequential()
        mlpmodel.add(Dense(275, input_shape=(X_train.shape[1],)))
        mlpmodel.add(Dense(50))
        mlpmodel.add(BatchNormalization())
        mlpmodel.add(Activation('relu'))
        mlpmodel.add(Dense(150))
        mlpmodel.add(BatchNormalization())
        mlpmodel.add(Activation('relu'))
        mlpmodel.add(Dense(900))
        mlpmodel.add(BatchNormalization())
        mlpmodel.add(Activation('relu'))
        mlpmodel.add(Dense(400))
        mlpmodel.add(BatchNormalization())
        mlpmodel.add(Activation('relu'))
        mlpmodel.add(Dense(7, activation='softmax'))
        
        return mlpmodel

    mlp = mlpmodel()
    

    mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min',min_delta=0.001, restore_best_weights=True)
    history = mlp.fit(X_train, y_train1, validation_data=(X_val, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)
    # Predict the test data
    y_pred = mlp.predict(X_test1)

    # Convert the predicted probabilities to class labels
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Create the confusion matrix
    cm = confusion_matrix(y_test, y_pred_labels)
    print("Confusion Matrix:")
    print(cm)

    # Calculate the F1 score
    f1 = f1_score(y_test, y_pred_labels, average='weighted')
    print("F1 Score for value of " + str(x) + ":", f1)

    f1_array.append((f1, x))
print(f1_array)

Epoch 1/50
416/416 [==============================] - 8s 13ms/step - loss: 0.3340 - accuracy: 0.8872 - val_loss: 0.6669 - val_accuracy: 0.7806
Epoch 2/50
416/416 [==============================] - 5s 12ms/step - loss: 0.1837 - accuracy: 0.9373 - val_loss: 0.1335 - val_accuracy: 0.9591
Epoch 3/50
416/416 [==============================] - 5s 12ms/step - loss: 0.1453 - accuracy: 0.9494 - val_loss: 0.1075 - val_accuracy: 0.9633
Epoch 4/50
416/416 [==============================] - 5s 11ms/step - loss: 0.1244 - accuracy: 0.9580 - val_loss: 0.1012 - val_accuracy: 0.9693
Epoch 5/50
416/416 [==============================] - 5s 11ms/step - loss: 0.1181 - accuracy: 0.9588 - val_loss: 0.1073 - val_accuracy: 0.9684
Epoch 6/50
416/416 [==============================] - 4s 11ms/step - loss: 0.0980 - accuracy: 0.9656 - val_loss: 0.1058 - val_accuracy: 0.9669
Epoch 7/50
416/416 [==============================] - 5s 11ms/step - loss: 0.0876 - accuracy: 0.9696 - val_loss: 0.0798 - val_accuracy: 0.9792

In [21]:
f1_df = pd.DataFrame(f1_array, columns=['F1 Score', 'Multiplier'])
f1_df.to_csv('f1_array.csv', index=False)
